In [0]:
import pyspark.sql.functions as f
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
inclusion_list = [
"SERETIDE"    ,"aaiat" ,"otitis"   ,"airway"   ,"allerg" ,            
  "antibiotics"   , "asthma"   ,"adherence"  ,     "atectura"    , "beclomethasone" ,     
 "breath"   ,"broncho"   ,"budesonide"  ,"calpol"   ,"chest pain",          
 "chiesi"   ,"combination therapy" ,"copd"   ,"corticosteroid"   ,    "cough"     ,          
 "covid" ,"dpi"  , "ellipta"        ,      "ers"        ,          "exacerbation"   ,     
 "flixotide"   ,         "fluticasone"     ,     "formoterol"     ,      "fostair"         ,     "foster"   ,       
 "gina"   ,"hospitalization"   ,"indacaterol"    ,"indications"   ,"infections" ,         
 "inhal" ,"innovair"  , "irritants"    , "laba"   ,"lama"   ,             
 "lft"    , "lung"   ,"maintenance"      ,    "mdi"       ,           "mometasone"  ,        
 "pediatric"     ,       "phid-cv"       ,       "pneumo"   ,"pulmon"   ,"reliever"    ,        
 "respiratory"  ,        "rhin"   ,"saba"   ,"salbutamol"   ,        "salmeterol"     ,     
 "symbicort"       ,     "triple"   ,"umeclidinium"    ,     "vaccine"        ,      "ventolin"  ,          
 "vilanterol", "wheez" ,"zinnat",     "zeneca",     "adolescen", 
 "chest" ,"child" ,"dtpa",       "ers",       "exacerbat", 
 "hexavalent", "ics",        "immun" ,"infan" ,"infect",    
 "maintenance therapy" , "mening",     "nucala",     "pediatr",    "prn",       
 "sirius",     "tdap",       "trelegy",    "triple therapy"       "tuberculosis"   , "wheeze","seretide"
]

In [0]:
exclusion_list = [
"403" , "404" , "academy" , "account" , "admin" , "adverse" , "aids" , "air" , "alopecia" , "atopic" , "ats" , 
  "attributes" , "aua" , "bacterial" , "bic/ftc/taf" , "bipolar" , "bph" , "broadcast" , "campaign" , 
  "cardiovascular" , "chat" , "check" , "in" , "checkout" , "citizens" , "classic" , "clinical" , "communication" ,
  "comorbidities" , "connect" , "connected" , "consent" , "contact" , "course" , "delivery" , "derma" , "details" ,
  "diabet" , "diagnosis" , "digipure" , "disease" , "dolutegravir" , "drugs ve tic. ace" , "dutasteride",
   "ecosystem" , "editorials" , "education" , "ehealth" , "endocrinology" , "engin" , "enrollment" , 
  "epilepsy","episode" , "error" , "ethics" , "event" , "expert" , "experts" , "first" , "guidelines" , "health" ,
  "hepatitis" , "highlight" , "history" , "hiv" , "home" , "infographic" , "inspiration" , "interview" , "invite" , "itu" , "join" , "log" , "in" , "login" , "luts" , "management" , "map" , "materials" , "medical" , "medline" , 
  "metabolism" , "module" , "N/A" , "online" , "order" , "Page" , "not" , "found" , "panorama" , 
  "participate","password" , "patients" , "paxil" , "practice" , "pregnant" , "press" , "releases" , "products" ,
  "professional" , "profile" , "psa" , "psychiatric" , "quality" , "register" , "remote" , "meeting" , "request" , 
  "resources" , "save" , "the" , "date" , "scientific" , "search" , "sexual" , "signup" , "skin" , "symptom" , 
  "therapeutic" , "thyroiditis" , "training" , "treatment" , "turkey" , "unsubscribe" , "updates" , "vaccin" , 
  "video" , "webcast" , "webinar" 
]

In [0]:
def remove_duplicates(duplist):
  """Method to remove duplicates"""
  if len(duplist) !=0:
    duplist = [set(duplist)]
  return duplist

def create_vocab(ngram1,ngram2,max_df,min_df):
  """Create vocabulary for all the data"""
  return CountVectorizer(ngram_range=(ngram1,ngram2),max_df=max_df, min_df=min_df, stop_words='english')#bigram,trigram
  #only include those words that appear in less than 80% of the document and appear in at least 2 documents,remove stopwords.Include unigram to pentagram.
  
def create_docterm_matrix(count_vectorizer,subject):
  """Create document-term matrix"""  
  return count_vectorizer.fit_transform(subject.values.astype('U'))
  
def perform_lda(n_topics,random_state):
  """Perform LDA on document-term matrix"""  
  return LatentDirichletAllocation(n_components=n_topics, random_state=random_state)

def top_ngrams(lda_object,num_words,count_vect,speciality):
  """Find top n-grams in each topic"""  
  for i,topic in enumerate(lda_object):
      print("Top words for topic #%d in %s:" %(i,speciality))
      subs=[count_vect.get_feature_names()[i] for i in topic.argsort()[-num_words:]]
      list1=[]
      for j in subs:
        flag=0        #Exclusion
        for k in exclusion_list:
          if k  in j:  
            flag=1
            break
        if flag==0:
          list1.append(j) 
        for l in inclusion_list: #Exclusion
          if l  in j:  
            list1.append(j)
            break    
      list1=remove_duplicates(list1)
      print(list1)
      print("\n")

def email_openrate(subject,speciality,open_count,min_email_sent):
  """Find top Email subjects lines with high Open Rate """ 
  speciality_emails_sent= speciality.groupBy(subject).count().orderBy('count',ascending=False).selectExpr(subject,'count as count_Email_sent')
  speciality_emails_opened= speciality.filter(f.col(open_count)>0).groupBy(subject).count().orderBy('count',ascending=False).selectExpr(subject,'count as count_Email_opened')
  speciality_email_open_rate = speciality_emails_opened.join(speciality_emails_sent,[subject],"inner")
  speciality_email_open_rate =speciality_email_open_rate.withColumn("OpenRate",f.col('count_Email_opened')/f.col('count_Email_sent')).orderBy('OpenRate',ascending=False).filter(f.col("count_Email_sent")>=min_email_sent) #To eliminate cases when #Emails sent is very less
  display(speciality_email_open_rate)

## 1-to-1 Email

In [0]:
#Reading the data
data1 = spark.sql("select * from seretide_topic_modelling.email")
#data1.fillna(0,subset=None)
display(data1)

Speciality_Segment,Account_vod__c,Subject_Line,Translated_Subject_Line,Email_Config_Values_vod__c,Product_vod__c_Name,Click_Count_vod__c,Open_Count_vod__c,Country_name,Email_Sent_Date_vod__c
Pediatricians,0016100000Us95vAAB,ERS 2021 Invitación a la presentación del estudio APPaRENT,ERS 2021 Invitation to the presentation of the APPaRENT Study,"{""Subject"":""ERS 2021: Invitación a la presentación del estudio APPaRENT"",""default_consent"":""Explicit_vod"",""User.Email"":""zuriel.a.najera@gsk.com"",""ReplyToName"":""Zuriel Alejandro Nájera Domínguez"",""FromName"":""Zuriel Alejandro Nájera Domínguez"",""EmailDomain"":""m.gsk.com"",""accFname"":""Marcelo"",""FromAddress"":""zuriel.a.najera@gsk.com"",""customTextArray"":[],""consent_level"":""Product_vod"",""test_mode"":""False"",""User.Title"":"""",""userName"":""Zuriel Alejandro Nájera Domínguez"",""User.MobilePhone"":"""",""customRichTextArray"":[],""ReplyTo"":""zuriel.a.najera@gsk.com"",""check_consent"":"""",""accLname"":""Valdés Garza""}",Seretide [MX-815],0.0,1.0,Mexico,2021-09-10 00:45:38
Pediatricians,0016100000Us95vAAB,ERS 2021 Invitación a la presentación del estudio APPaRENT,ERS 2021 Invitation to the presentation of the APPaRENT Study,"{""Subject"":""ERS 2021: Invitación a la presentación del estudio APPaRENT"",""default_consent"":""Explicit_vod"",""User.Email"":""zuriel.a.najera@gsk.com"",""ReplyToName"":""Zuriel Alejandro Nájera Domínguez"",""FromName"":""Zuriel Alejandro Nájera Domínguez"",""EmailDomain"":""m.gsk.com"",""accFname"":""Marcelo"",""FromAddress"":""zuriel.a.najera@gsk.com"",""customTextArray"":[],""consent_level"":""Product_vod"",""test_mode"":""False"",""User.Title"":"""",""userName"":""Zuriel Alejandro Nájera Domínguez"",""User.MobilePhone"":"""",""customRichTextArray"":[],""ReplyTo"":""zuriel.a.najera@gsk.com"",""check_consent"":"""",""accLname"":""Valdés Garza""}",Seretide [MX-815],0.0,1.0,Mexico,2021-09-10 00:45:38
Pediatricians,0016100000Us95vAAB,ERS 2021 Invitación a la presentación del estudio APPaRENT,ERS 2021 Invitation to the presentation of the APPaRENT Study,"{""Subject"":""ERS 2021: Invitación a la presentación del estudio APPaRENT"",""default_consent"":""Explicit_vod"",""User.Email"":""zuriel.a.najera@gsk.com"",""ReplyToName"":""Zuriel Alejandro Nájera Domínguez"",""FromName"":""Zuriel Alejandro Nájera Domínguez"",""EmailDomain"":""m.gsk.com"",""accFname"":""Marcelo"",""FromAddress"":""zuriel.a.najera@gsk.com"",""customTextArray"":[],""consent_level"":""Product_vod"",""test_mode"":""False"",""User.Title"":"""",""userName"":""Zuriel Alejandro Nájera Domínguez"",""User.MobilePhone"":"""",""customRichTextArray"":[],""ReplyTo"":""zuriel.a.najera@gsk.com"",""check_consent"":"""",""accLname"":""Valdés Garza""}",Seretide [MX-815],0.0,1.0,Mexico,2021-09-10 00:45:38
Pediatricians,0016100000Us5IzAAJ,ERS 2021 Invitación a la presentación del estudio APPaRENT,ERS 2021 Invitation to the presentation of the APPaRENT Study,"{""Subject"":""ERS 2021: Invitación a la presentación del estudio APPaRENT"",""default_consent"":""Explicit_vod"",""User.Email"":""anaid.a.sandoval@gsk.com"",""ReplyToName"":""Anaid Arely Sandoval Acevedo"",""FromName"":""Anaid Arely Sandoval Acevedo"",""EmailDomain"":""m.gsk.com"",""accFname"":""Rafael"",""FromAddress"":""anaid.a.sandoval@gsk.com"",""customTextArray"":[],""consent_level"":""Product_vod"",""test_mode"":""False"",""User.Title"":"""",""userName"":""Anaid Arely Sandoval Acevedo"",""User.MobilePhone"":"""",""customRichTextArray"":[],""ReplyTo"":""anaid.a.sandoval@gsk.com"",""check_consent"":"""",""accLname"":""López Bautista""}",Seretide [MX-815],0.0,2.0,Mexico,2021-09-10 21:32:25
Pediatricians,0016100000Us5IzAAJ,ERS 2021 Invitación a la presentación del estudio APPaRENT,ERS 2021 Invitation to the presentation of the APPaRENT Study,"{""Subject"":""ERS 2021: Invitación a la presentación del estudio APPaRENT"",""default_consent"":""Explicit_vod"",""User.Email"":""anaid.a.sandoval@gsk.com"",""ReplyToName"":""Anaid Arely S

In [0]:
display(spark.sql("select distinct Speciality_Segment  from seretide_topic_modelling.email"))

Speciality_Segment
General Practitioners
Pediatricians
Pulmonologists
Pharmacists


In [0]:
display(spark.sql("select distinct Country_name  from seretide_topic_modelling.email"))

Country_name
Argentina
Mexico
Brazil
Thailand
Turkey


In [0]:
speciality1=spark.sql("select Translated_Subject_Line,Open_Count_vod__c  from seretide_topic_modelling.email where Speciality_Segment='General Practitioners'")
speciality2=spark.sql("select Translated_Subject_Line,Open_Count_vod__c  from seretide_topic_modelling.email where Speciality_Segment='Pediatricians'")
speciality3=spark.sql("select Translated_Subject_Line,Open_Count_vod__c  from seretide_topic_modelling.email where Speciality_Segment='Pharmacists'")
speciality4=spark.sql("select Translated_Subject_Line,Open_Count_vod__c  from seretide_topic_modelling.email where Speciality_Segment='Pulmonologists'")

In [0]:
speciality1_pd=speciality1.toPandas()
speciality2_pd=speciality2.toPandas()
speciality3_pd=speciality3.toPandas()
speciality4_pd=speciality4.toPandas()

####Create vocabulary for all the data for each Speciality and also document-term matrix

In [0]:
count_vect1 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect2 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect3 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect4 = create_vocab(2,3,0.8,2)#bigram,trigram
#only include those words that appear in less than 80% of the document and appear in at least 2 documents,remove stopwords.Include unigram to pentagram.
doc_term_matrix1 = create_docterm_matrix(count_vect1,speciality1_pd['Translated_Subject_Line'])
doc_term_matrix2 = create_docterm_matrix(count_vect2,speciality2_pd['Translated_Subject_Line'])
doc_term_matrix3 = create_docterm_matrix(count_vect3,speciality3_pd['Translated_Subject_Line'])
doc_term_matrix4 = create_docterm_matrix(count_vect4,speciality4_pd['Translated_Subject_Line'])

####Perform LDA on our document-term matrix

In [0]:
#Have 4 topics
LDA1=  perform_lda(4,42)
LDA2 = perform_lda(4,42)
LDA3 = perform_lda(4,42)
LDA4 = perform_lda(4,42)
LDA1.fit(doc_term_matrix1)
LDA2.fit(doc_term_matrix2)
LDA3.fit(doc_term_matrix3)
LDA4.fit(doc_term_matrix4)

Out[86]: LatentDirichletAllocation(n_components=4, random_state=42)

####Find top  n-grams in each topic and Speciality

In [0]:
top_ngrams(LDA1.components_,5,count_vect1,'General Practitioners')

Top words for topic #0 in General Practitioners:
[{'asthma control', 'patients asthma', 'asthma patients', 'patients asthma control'}]


Top words for topic #1 in General Practitioners:
[{'proactive asthma control', 'proactive asthma', 'asthma control'}]


Top words for topic #2 in General Practitioners:
[{'asthma day', 'asthma control'}]


Top words for topic #3 in General Practitioners:
[{'apparent study perception', 'asthma control', 'new apparent study', 'aligned reality', 'apparent study'}]

In [0]:
top_ngrams(LDA2.components_,5,count_vect2,'Pediatricians')

Top words for topic #0 in Pediatricians:
[{'cycle 10', 'proactive controlling asthma', 'asthma seretide', 'controlling asthma seretide'}]


Top words for topic #1 in Pediatricians:
[{'asthma control', 'asthma patients', 'proactive asthma control', 'step proactive asthma', 'proactive asthma'}]


Top words for topic #2 in Pediatricians:
[{'achieve asthma control', 'asthma treatment', 'asthma control', 'achieve asthma', 'asthma symptoms protects'}]


Top words for topic #3 in Pediatricians:
[{'discount viver', 'asthma control', 'viver mais', '46 31', 'seretide discount'}]

In [0]:
top_ngrams(LDA3.components_,5,count_vect3,'Pharmacists')

Top words for topic #0 in Pharmacists:
[{'copd patients', 'hospitalised copd patients'}]


Top words for topic #1 in Pharmacists:
[{'moderate asthma management', 'cornerstone moderate', 'asthma management', 'moderate asthma', 'cornerstone moderate asthma'}]


Top words for topic #2 in Pharmacists:
[{'copd situation', 'copd situation covid', 'situation covid 19', 'manage asthma copd', 'asthma copd situation'}]


Top words for topic #3 in Pharmacists:
[{'treatment seretide', 'copd patients proactive', 'help copd'}]

In [0]:
top_ngrams(LDA4.components_,5,count_vect4,'Pulmonologists')

Top words for topic #0 in Pulmonologists:
[{'know apparent', 'know apparent study', 'asthma control', 'ers 2021', 'apparent study'}]


Top words for topic #1 in Pulmonologists:
[{'cycle 10', 'asthma control', 'patient risk', 'risk exacerbations', 'reduce patient'}]


Top words for topic #2 in Pulmonologists:
[{'protects airways', 'molecules relieves', 'asthma control'}]


Top words for topic #3 in Pulmonologists:
[{'asthma control', 'seretide proactive asthma', 'proactive asthma control', 'proactive asthma', 'seretide proactive'}]

### Find Subject lines for 1-to-1-Email with high Open Rate for each Speciality

In [0]:
print("############################General Practitioners####################")
email_openrate("Translated_Subject_Line",speciality1,"Open_Count_vod__c",10)

############################General Practitioners####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
Asthma control assessment,486,636,0.7641509433962265
The cornerstone of moderate asthma management,563,765,0.7359477124183007
Short Clip VDO_Asthma Case Sharing CU-MAR 1,54,77,0.7012987012987013
ขอเชิญรับชม rerun COPD Symposium งานประชุม TST Masterclass 2022,44,66,0.6666666666666666
ขอเชิญรับชม Short VDO COPD Content - Benefit-Risk balance of ICS-containing in COPD,32,48,0.6666666666666666
Should asthma treatment be stepped down?,91,139,0.6546762589928058
You’re Invited to our Asthma Day Webinar,180,278,0.6474820143884892
Power of proactive regular dosing in mild asthma,251,389,0.6452442159383034
The stepwise approach in asthma management,46,72,0.6388888888888888
Short Clip VDO Don't compromise Asthma control Treat Airway Inflammation,14,22,0.6363636363636364


In [0]:
print("############################Pediatricians####################")
email_openrate("Translated_Subject_Line",speciality2,"Open_Count_vod__c",10)

############################Pediatricians####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
Asthma patients perception vs reality,30,32,0.9375
Pediatric asthma management in mild asthma,167,233,0.7167381974248928
Pediatric asthma management in moderate asthma,80,113,0.7079646017699115
Asthma control assessment,80,119,0.6722689075630253
"In this session, Prof.Sumalee will discuss on the definition of asthma control and how to achevie asthma control ?",13,20,0.65
From evidenct to practice in mild asthma,729,1196,0.6095317725752508
Keep control of your patients during the pandemic,14,23,0.6086956521739131
Power of proactive regular dosing in mild asthma,110,182,0.6043956043956044
The cornerstone of moderate asthma management,83,138,0.6014492753623188
Short Clip VDO_Asthma Case Sharing CU-MAR 1,43,72,0.5972222222222222


In [0]:
print("############################Pharmacists####################")
email_openrate("Translated_Subject_Line",speciality3,"Open_Count_vod__c",1)

############################Pharmacists####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
Myths and truths - treatment of hospitalised COPD patients,2,2,1.0
Short Clip VDO by Rep_Dr.Siwasak Juthong How do we know Who has asthma symptom?,2,2,1.0
Benefits of using Seretide and Vanisto,1,1,1.0
The cornerstone of moderate asthma management,6,8,0.75
Help your COPD patients with proactive treatment with Seretide,2,3,0.6666666666666666
How to manage asthma & COPD in situatioN of COVID-19,1,5,0.2


In [0]:
print("############################Pulmonologists####################")
email_openrate("Translated_Subject_Line",speciality4,"Open_Count_vod__c",10)

############################Pulmonologists####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
ขอเชิญรับชม rerun COPD Symposium งานประชุม TST Masterclass 2022,14,18,0.7777777777777778
Seretide Channel Frame - Veeva Emails_Argentina,13,17,0.7647058823529411
Asthma control assessment,170,232,0.7327586206896551
Asthma patients perception vs reality,82,114,0.7192982456140351
Key Points for the Management of Patients with COPD during COVID-19 pandemic,50,70,0.7142857142857143
The cornerstone of moderate asthma management,248,356,0.6966292134831461
Veeva Email Invitación para Asthma Management Keeping it Personal Weinar 21 abril MX,143,216,0.6620370370370371
"What is proactive, sustained asthma control?",34,52,0.6538461538461539
Short Clip VDO_Asthma Case Sharing CU-MAR 1,48,74,0.6486486486486487
5 Mins for COPD - Factor to consider ICS/LABA in COPD,50,78,0.6410256410256411


## Mass Email

In [0]:
#Reading the data
data2 = spark.sql("select * from seretide_topic_modelling.mass_email")
#data1.fillna(0,subset=None)
display(data2)

Speciality_Segment,User_ID,Contact_Date,Campaign_Subject_Line,Translated_Subject_Line,Campaign_Name,Email_Opened,Email_Clicked,Brand,Country_Name
Pediatricians,0CAB0902-3087-4BCE-90E2-E91FB5C409BB,2022-02-12T01:40:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,0,SERETIDE,Mexico
Pediatricians,0CAB0902-3087-4BCE-90E2-E91FB5C409BB,2022-02-12T01:40:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,0,SERETIDE,Mexico
Pediatricians,0CAB0902-3087-4BCE-90E2-E91FB5C409BB,2022-02-12T01:40:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,1,0,SERETIDE,Mexico
Pediatricians,0CAB0902-3087-4BCE-90E2-E91FB5C409BB,2022-02-12T01:40:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,1,0,SERETIDE,Mexico
Pediatricians,0CAB0902-3087-4BCE-90E2-E91FB5C409BB,2022-02-12T01:40:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,1,SERETIDE,Mexico
Pediatricians,0CAB0902-3087-4BCE-90E2-E91FB5C409BB,2022-02-12T01:40:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,1,SERETIDE,Mexico
Pediatricians,6817C112-1AC6-465E-806D-6F56E46BBBC3,2022-02-23T14:00:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,1,0,SERETIDE,Mexico
Pediatricians,6817C112-1AC6-465E-806D-6F56E46BBBC3,2022-02-23T14:00:49.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,0,SERETIDE,Mexico
Pediatricians,3B6021A3-1FA0-46E0-828C-FC106EB0DABA,2022-02-17T12:46:30.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,0,SERETIDE,Mexico
Pediatricians,3B6021A3-1FA0-46E0-828C-FC106EB0DABA,2022-02-17T12:46:30.000+0000,¡Gracias por registrarse a nuestro webcast!,Thank you for registering for our webcast!,null,0,0,SERETIDE,Mexico


In [0]:
display(spark.sql("select distinct Speciality_Segment  from seretide_topic_modelling.mass_email"))

Speciality_Segment
Pediatricians
Pulmonologists
General Practitioners


In [0]:
speciality1=spark.sql("select Translated_Subject_Line,Email_Opened  from seretide_topic_modelling.mass_email where Speciality_Segment='General Practitioners'")
speciality2=spark.sql("select Translated_Subject_Line,Email_Opened  from seretide_topic_modelling.mass_email where Speciality_Segment='Pediatricians'")
speciality3=spark.sql("select Translated_Subject_Line,Email_Opened  from seretide_topic_modelling.mass_email where Speciality_Segment='Pulmonologists'")

In [0]:
speciality1_pd=speciality1.toPandas()
speciality2_pd=speciality2.toPandas()
speciality3_pd=speciality3.toPandas()

####Create vocabulary for all the data for each Speciality and also document-term matrix

In [0]:
count_vect1 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect2 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect3 = create_vocab(2,3,0.8,2)#bigram,trigram
#only include those words that appear in less than 80% of the document and appear in at least 2 documents,remove stopwords.Include unigram to pentagram.
doc_term_matrix1 = create_docterm_matrix(count_vect1,speciality1_pd['Translated_Subject_Line'])
doc_term_matrix2 = create_docterm_matrix(count_vect2,speciality2_pd['Translated_Subject_Line'])
doc_term_matrix3 = create_docterm_matrix(count_vect3,speciality3_pd['Translated_Subject_Line'])

####Perform LDA on our document-term matrix

In [0]:
#Have 4 topics
LDA1=  perform_lda(4,42)
LDA2 = perform_lda(4,42)
LDA3 = perform_lda(4,42)
LDA1.fit(doc_term_matrix1)
LDA2.fit(doc_term_matrix2)
LDA3.fit(doc_term_matrix3)

Out[100]: LatentDirichletAllocation(n_components=4, random_state=42)

####Find top  n-grams in each topic and Speciality

In [0]:
top_ngrams(LDA1.components_,5,count_vect1,'General Practitioners')

Top words for topic #0 in General Practitioners:
[{'sep_vdo webcast asthma_practical', 'goal asthma management'}]


Top words for topic #1 in General Practitioners:
[{'news asthma treatment', 'asthma treatment', 'asthma treatment strategies', 'news asthma'}]


Top words for topic #2 in General Practitioners:
[{'change treat'}]


Top words for topic #3 in General Practitioners:
[{'asthma กษาง', 'asthma กษาง ายๆแต', 'newcomer asthma easy', 'newcomer asthma', 'eacc newcomer'}]

In [0]:
top_ngrams(LDA2.components_,5,count_vect2,'Pediatricians')

Top words for topic #0 in Pediatricians:
[{'abcs asthma', 'abc asthma diagnosis', 'abc asthma', 'asthma diagnosis', 'abcs asthma diagnosis'}]


Top words for topic #1 in Pediatricians:
[{'gsk gaa', 'gaa global'}]


Top words for topic #2 in Pediatricians:
[{'legacy trust', 'seretide legacy trust', 'seretide legacy'}]


Top words for topic #3 in Pediatricians:
[{'gsk wishes', 'wishes happy holidays', 'gsk wishes happy', 'wishes happy', 'happy holidays'}]

In [0]:
top_ngrams(LDA3.components_,5,count_vect3,'Pulmonologists')

Top words for topic #0 in Pulmonologists:
[{'seretide legacy trust', 'seretide legacy', 'abc asthma', 'asthma diagnosis', 'legacy trust'}]


Top words for topic #1 in Pulmonologists:
[{'let child asthma', 'let child', 'asthma window', 'child asthma', 'child asthma window'}]


Top words for topic #2 in Pulmonologists:
[{'gsk wishes', 'wishes happy holidays', 'gsk wishes happy', 'wishes happy', 'happy holidays'}]


Top words for topic #3 in Pulmonologists:
[{'abcs asthma diagnosis', 'asthma diagnosis', 'webcast abcs asthma', 'abcs asthma'}]

### Find Subject lines for 1-to-1-Email with high Open Rate for each Speciality

In [0]:
print("############################General Practitioners####################")
email_openrate("Translated_Subject_Line",speciality1,"Email_Opened",10)

############################General Practitioners####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
6-12 Sep_VDO Webcast Asthma_Practical points to achieve goal of asthma management,1000,2440,0.4098360655737705
Invite GSK Symposium - Introduction to chest medicine 2021,941,2336,0.4028253424657534
PRIMER Module 3 KIDS is now available!,18,45,0.4
Invite to watch - Rerun COPD Symposium from European Respiratory Society 2021,937,2399,0.3905794080867028
The second PRIMER module is coming!,56,153,0.3660130718954248
PRIMER Asthma and COPD Module 3 is now available!,30,85,0.35294117647058826
EACC Newcomer Asthma : Easy Asthma รักษาง่ายๆแต่ได้คุณภาพ,720,2115,0.3404255319148936
Register for our webinar From asthma guidelines to my practice: What else can we do to improve control?,61,190,0.32105263157894737
The second PRIMER module is now available!,44,140,0.3142857142857143
GSK invites you to World Asthma Day 2022,111,360,0.30833333333333335


In [0]:
print("############################Pediatricians####################")
email_openrate("Translated_Subject_Line",speciality2,"Email_Opened",10)

############################Pediatricians####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
Thank you for attending our webcast!,22,41,0.5365853658536586
You are already registered for our webinar!,34,65,0.5230769230769231
We regret that you were unable to attend our webinar.,23,44,0.5227272727272727
Thank you for registering for our webcast!,140,272,0.5147058823529411
"""What will we talk about in the webinar """"Back to ABC: Asthma Under Control in Children""""?",22,43,0.5116279069767442
Thank you for your attendance to our live streaming!,15,30,0.5
"The ABC's of Asthma Diagnosis, don't miss it!",127,254,0.5
Thank you for registering to our live stream!,46,100,0.46
PRIMER Asthma and COPD Module 3 is now available!,48,109,0.44036697247706424
6-12 Sep_VDO Webcast Asthma_Practical points to achieve goal of asthma management,51,116,0.4396551724137931


In [0]:
print("############################Pulmonologists####################")
email_openrate("Translated_Subject_Line",speciality3,"Email_Opened",10)

############################Pulmonologists####################

Translated_Subject_Line,count_Email_opened,count_Email_sent,OpenRate
Thank you for registering for the webinar on the latest advances in the therapeutic management of patients with asthma.,7,11,0.6363636363636364
Thank you for registering to our live stream!,18,32,0.5625
Thank you for attending our webcast!,12,23,0.5217391304347826
"The ABC's of Asthma Diagnosis, don't miss it!",58,115,0.5043478260869565
We regret that you were unable to attend our webinar.,14,28,0.5
"Seretide proactively prevents exacerbations, not just symptoms.",17,35,0.4857142857142857
6-12 Sep_VDO Webcast Asthma_Practical points to achieve goal of asthma management,26,54,0.48148148148148145
"""What will we talk about in the webinar """"Back to ABC: Asthma Under Control in Children""""?",10,21,0.47619047619047616
You are already registered for our webinar!,15,32,0.46875
Regular or flexible ICS regimens for asthma control?,14,31,0.45161290322580644


## Edetail

In [0]:
#Reading the data
data3 = spark.sql("select * from seretide_topic_modelling.edetail")
#data1.fillna(0,subset=None)
display(data3)

Speciality_Segment,Id,Product_vod__c,Product_vod__c_Name,Account_vod__c,Call2_vod__c,CLM_ID_vod__c,Clm_Presentation_Name_vod__c,Key_Message_Name_vod__c,Translated_Key_Message,Presentation_ID_vod__c,Duration_vod__c,Country_Name
Pediatricians,a080K00005bZRgUQAW,a002800000MTdT3AAL,SERETIDE ASTHMA,0010K00002DVec5QAD,a040K00002MnmXvQAJ,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_010.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005bWCjQQAW,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000ESs1xAAD,a040K00002Mm5hEQAR,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_010.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005bWCjQQAW,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000ESs1xAAD,a040K00002Mm5hEQAR,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_010.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005eMjIfQAK,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000efYnXAAU,a040K00002N8m6XQAR,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_010.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005l7dp0QAA,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000efYnXAAU,a040K00002OiEPeQAN,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_004.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,3.0,Thailand
Pediatricians,a080K00005eMjIZQA0,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000efYnXAAU,a040K00002N8m6XQAR,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_004.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005j2KjVQAU,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800001QVnEKAA1,a040K00002OFgZSQA1,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_004.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005j2KjVQAU,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800001QVnEKAA1,a040K00002OFgZSQA1,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_004.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,1.0,Thailand
Pediatricians,a080K00005dJa3NQAS,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000ESrPwAAL,a040K00002NAH20QAH,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_010.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,2.0,Thailand
Pediatricians,a080K00005dJa3NQAS,a002800000MTdT3AAL,SERETIDE ASTHMA,0012800000ESrPwAAL,a040K00002NAH20QAH,Portfolio_Paediatric_asthma_2021_TH1.0_TDTS_010.zip,Portfolio Paediatric asthma 2021 TH1.2 (TDTS),SAFETY,SAFETY,a110K000009p6aUQAQ,2.0,Thailand


In [0]:
display(spark.sql("select distinct Speciality_Segment  from seretide_topic_modelling.edetail"))

Speciality_Segment
General Practitioners
Pediatricians
Pulmonologists
Pharmacists


In [0]:
speciality1=spark.sql("select Translated_Key_Message  from seretide_topic_modelling.edetail where Speciality_Segment='General Practitioners'")
speciality2=spark.sql("select Translated_Key_Message  from seretide_topic_modelling.edetail where Speciality_Segment='Pediatricians'")
speciality3=spark.sql("select Translated_Key_Message  from seretide_topic_modelling.edetail where Speciality_Segment='Pharmacists'")
speciality4=spark.sql("select Translated_Key_Message  from seretide_topic_modelling.edetail where Speciality_Segment='Pulmonologists'")

In [0]:
speciality1_pd=speciality1.toPandas()
speciality2_pd=speciality2.toPandas()
speciality3_pd=speciality3.toPandas()
speciality4_pd=speciality4.toPandas()

####Create vocabulary for all the data for each Attitude and also document-term matrix

In [0]:
count_vect1 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect2 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect3 = create_vocab(2,3,0.8,2)#bigram,trigram
count_vect4 = create_vocab(2,3,0.8,2)#bigram,trigram
#only include those words that appear in less than 80% of the document and appear in at least 2 documents,remove stopwords.Include unigram to pentagram.
doc_term_matrix1 = create_docterm_matrix(count_vect1,speciality1_pd['Translated_Key_Message'])
doc_term_matrix2 = create_docterm_matrix(count_vect2,speciality2_pd['Translated_Key_Message'])
doc_term_matrix3 = create_docterm_matrix(count_vect3,speciality3_pd['Translated_Key_Message'])
doc_term_matrix4 = create_docterm_matrix(count_vect4,speciality4_pd['Translated_Key_Message'])

####Perform LDA on our document-term matrix

In [0]:
#Have 4 topics
LDA1=  perform_lda(4,42)
LDA2 = perform_lda(4,42)
LDA3 = perform_lda(4,42)
LDA4 = perform_lda(4,42)
LDA1.fit(doc_term_matrix1)
LDA2.fit(doc_term_matrix2)
LDA3.fit(doc_term_matrix3)
LDA4.fit(doc_term_matrix4)

Out[112]: LatentDirichletAllocation(n_components=4, random_state=42)

####Find top  n-grams in each topic and Speciality

In [0]:
top_ngrams(LDA1.components_,5,count_vect1,'General Practitioners')

Top words for topic #0 in General Practitioners:
[{'non brand', 'non brand objective', 'brand objective', '2021 th1'}]


Top words for topic #1 in General Practitioners:
[{'summary kc', 'seretide copd', 'exacerbation 2021', 'exacerbation 2021 th1', '2021 th1'}]


Top words for topic #2 in General Practitioners:
[{'seretide slide deck', 'slide deck', 'seretide slide', 'copd 2021 th1', '2021 th1'}]


Top words for topic #3 in General Practitioners:
[{'cornerstone asthma', 'asthma management', 'asthma management 2021', '2021 th1'}]

In [0]:
top_ngrams(LDA2.components_,5,count_vect2,'Pediatricians')

Top words for topic #0 in Pediatricians:
[{'non brand', 'non brand objective', 'brand objective'}]


Top words for topic #1 in Pediatricians:
[{'cornerstone asthma', 'asthma management', '2021 th1'}]


Top words for topic #2 in Pediatricians:
[{'seretide eda', 'eda prices', 'ped asthma', 'seretide eda prices', '2021 th1'}]


Top words for topic #3 in Pediatricians:
[{'asthma 2021 mx1', '2021 mx1', '007 popup', 'asthma 2021', 'slide deck'}]

In [0]:
top_ngrams(LDA3.components_,5,count_vect3,'Pharmacists')

Top words for topic #0 in Pharmacists:
[{'non brand', 'brand objective', 'seretide slide', 'non brand objective', '2021 th1'}]


Top words for topic #1 in Pharmacists:
[{'power ics', 'ics laba', '2021 th1 005', 'th1 005', '2021 th1'}]


Top words for topic #2 in Pharmacists:
[{'seretide asthama campaign', '2021 th1'}]


Top words for topic #3 in Pharmacists:
[{'accuhaler easy', 'version 2021', 'use version', 'version 2021 th1', 'use version 2021'}]

In [0]:
top_ngrams(LDA4.components_,5,count_vect4,'Pulmonologists')

Top words for topic #0 in Pulmonologists:
[{'non brand', 'non brand objective', 'asthma management 2021', 'brand objective'}]


Top words for topic #1 in Pulmonologists:
[{'asthma control', '2022 th1', 'asthma 2022 th1', 'seretide practical', 'asthma 2022'}]


Top words for topic #2 in Pulmonologists:
[{'seretide eda', 'slide deck', 'eda prices', 'seretide eda prices', '2021 th1'}]


Top words for topic #3 in Pulmonologists:
[{'ics laba', 'laba 2021 th1', 'apparent study', '2021 th1'}]

## Portal

In [0]:
#Reading the data
data4 = spark.sql("select * from seretide_topic_modelling.portal")
#data1.fillna(0,subset=None)
display(data4)

Speciality_Segment,post_evar75,post_evar30,post_evar32,Translated_Web_Page,Country_Name
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil
Pediatricians,9263514A-4299-4501-B077-0405731E442D,https://gskpro.com/pt-br/campanhas/lp-seretide.html?cc=br_ema_ac_pm-br-fps-eml-210118_59914&utm_campaign=acc+2022+-+onda+4+-+resp+seretide+ped&utm_content=seretide+%282%29&utm_medium=email&utm_source=emailmarketing&utm_term=como+seretide+auxilia+no+contro,seretide,seretide,Brazil


In [0]:
display(spark.sql("select distinct Speciality_Segment  from seretide_topic_modelling.portal"))

Speciality_Segment
Pediatricians
Pulmonologists


In [0]:
display(spark.sql("select distinct Translated_Web_Page from seretide_topic_modelling.portal where Speciality_Segment='Pediatricians'"))

Translated_Web_Page
materials of interest seretide
seretide 19-oct-21
seretide


In [0]:
display(spark.sql("select distinct Translated_Web_Page from seretide_topic_modelling.portal where Speciality_Segment='Pulmonologists'"))

Translated_Web_Page
seretide


In [0]:
speciality1=spark.sql("select Translated_Web_Page  from seretide_topic_modelling.portal where Speciality_Segment='Pediatricians'")
speciality2=spark.sql("select Translated_Web_Page  from seretide_topic_modelling.portal where Speciality_Segment='Pulmonologists'")


In [0]:
speciality1_pd=speciality1.toPandas()
speciality2_pd=speciality2.toPandas()

####Create vocabulary for all the data for each Attitude and also document-term matrix

In [0]:
count_vect1 = create_vocab(1,3,0.8,2)#bigram,trigram
count_vect2 = create_vocab(1,3,1.0,1)#bigram,trigram
doc_term_matrix1 = create_docterm_matrix(count_vect1,speciality1_pd['Translated_Web_Page'])
doc_term_matrix2 = create_docterm_matrix(count_vect2,speciality2_pd['Translated_Web_Page'])

####Perform LDA on our document-term matrix

In [0]:
#Have 4 topics
LDA1=  perform_lda(4,42)
LDA2 = perform_lda(4,42)
LDA1.fit(doc_term_matrix1)
LDA2.fit(doc_term_matrix2)

Out[124]: LatentDirichletAllocation(n_components=4, random_state=42)

####Find top  n-grams in each topic and Speciality

In [0]:
top_ngrams(LDA1.components_,5,count_vect1,'Pediatricians')

Top words for topic #0 in Pediatricians:
[{'materials seretide', 'seretide 19 oct', 'oct 21', 'seretide 19'}]


Top words for topic #1 in Pediatricians:
[{'oct 21', 'oct', 'seretide 19', 'seretide 19 oct', '21'}]


Top words for topic #2 in Pediatricians:
[{'oct 21', 'oct', 'seretide 19 oct', '21', '19'}]


Top words for topic #3 in Pediatricians:
[{'oct 21', 'oct', 'seretide 19', 'seretide 19 oct', '21'}]

In [0]:
top_ngrams(LDA2.components_,5,count_vect2,'Pulmonologists')

Top words for topic #0 in Pulmonologists:
[{'seretide'}]


Top words for topic #1 in Pulmonologists:
[{'seretide'}]


Top words for topic #2 in Pulmonologists:
[{'seretide'}]


Top words for topic #3 in Pulmonologists:
[{'seretide'}]